In [ ]:
# This part of the analysis should be done on wynton

molecule_info_path = 'molecule_info.h5'


# Function to print all datasets in an HDF5 file
def print_datasets(name, obj):
    if isinstance(obj, h5py.Dataset):
        print(name)
        
    
        
with h5py.File(molecule_info_path, 'r') as f:
    # Extract gRNA information. Adjust the keys as necessary based on your Cell Ranger version
    feature_ids = f['features/id'][:]
    feature_types = f['features/feature_type'][:]
    cell_barcodes = f['barcodes'][:]
    umi_counts = f['umi'][:]
    barcode_idx = f['barcode_idx'][:]
    feature_idx = f['feature_idx'][:]
    sequence = f['features/sequence'][:]


cell_barcodes = pd.DataFrame({
        'cell_barcodes': cell_barcodes
    })


# Resetting the index and renaming the index column
cell_barcodes = cell_barcodes.reset_index()
cell_barcodes = cell_barcodes.rename(columns={'index': 'barcode_idx'})


feature_id_sequences = pd.DataFrame({
    'sequence': sequence,
    'feature_ids': feature_ids
})


# Resetting the index and renaming the index column
feature_id_sequences = feature_id_sequences.reset_index()
feature_id_sequences = feature_id_sequences.rename(columns={'index': 'feature_idx'})

feature_id_sequences = feature_id_sequences[feature_id_sequences['sequence'] != b'']

barcode_data = pd.DataFrame({
    'barcode_idx': barcode_idx,
    'feature_idx': feature_idx,
    'umi_counts': umi_counts
})

# Merge the dataframes
merged_data = barcode_data.merge(feature_id_sequences, on='feature_idx', how='left')

merged_data = merged_data.merge(cell_barcodes, on='barcode_idx', how='left')

filtered_data = merged_data.dropna()

filtered_data.to_csv('OS384_molecular_data.csv', index=False)